### Linear Regression
** Modeling a Linear Relationship in Tensorflow **  
In this notebook we model the relationship between ground temperature and the chirping of crickets. The dataset is derived from a book "Songs of the Insects" written in 1948 by  G. W. Pierce.

In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Download data
!wget -nv -O data/PierceCricketData.csv 'https://ibm.box.com/shared/static/reyjo1hk43m2x79nreywwfwcdd5yi8zu.csv'

In [2]:
datafile = pd.read_csv("data/PierceCricketData.csv")
datafile.head()

,Chirps,Temp
0,20.0,88.6
1,16.0,71.6
2,19.8,93.3
3,18.4,84.3
4,17.1,80.6
